In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Polygon, Circle
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib.collections import PolyCollection

import numpy as np
from sklearn.cluster import KMeans
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras.losses import Loss
from tensorflow.keras.utils import load_img
import pprint as pp

from src.utils.box_cutter import BoundingBox_Processor
from src.utils.classes import CategoricalDataGen
from src.utils.funcs import init_COCO, make_predictions

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2
%aimport src.utils.funcs
%aimport src.utils.classes

In [ ]:
pp.PrettyPrinter(indent=4)

In [ ]:
data = init_COCO("./data/", ['train', 'val', 'test'])

In [ ]:
train = CategoricalDataGen(data_name='train',
                           coco_obj=data,
                           image_path='./data/images/',
                           target_size=(384, 512))

In [2]:
def generate_anchors(labels:tf.Tensor, boxes_per_cell:int=3, **kwargs):
    # flatten labels
    input_shape = labels.shape
    batch, xdiv, ydiv, _ = input_shape
    box_labels = tf.reshape(labels, [batch * xdiv * ydiv, 19])[..., 14:].numpy()
    box_labels[:, 0:1] = 0 
    box_labels[:, 1:2] = 0 
    # mask no object
    mask_labels = tf.cast(tf.reduce_sum(box_labels, axis=-1) > .001, dtype=tf.bool)
    box_labels = tf.boolean_mask(box_labels, mask_labels, axis=0).numpy()
    # find clusters
    clusters = KMeans(n_clusters=boxes_per_cell, max_iter=100, **kwargs)
    clusters.fit(box_labels)
    # retrieve the cluster space mapping
    cls = clusters.predict(box_labels)
    cls = np.expand_dims(cls, axis=-1)
    # The idea here is to pull the centroids out of the KMeans object by predicting on the labels
    # and then looping through the class predictions and taking the mean of each point in the class
    centroid_locations = np.ones((13,), dtype=np.float32)
    for idx in range(boxes_per_cell):
        filter = tf.where(tf.equal(cls, idx),
                                 box_labels,
                                 np.zeros(box_labels.shape, dtype=np.float32))
        mask = tf.cast(tf.abs(tf.reduce_sum(filter, axis=-1)) > .001, dtype=tf.float32)
        average = np.append(np.ones((1,), dtype=np.float32), tf.reduce_sum(filter, axis=0) / tf.reduce_sum(mask, axis=0))
        centroid_locations = np.append(centroid_locations, average)
  
    idx_template = np.fromfunction(lambda x, y: [x, y], (xdivs, ydivs), dtype=np.float32)   
    knudge_coords = (np.stack((idx_template[0], idx_template[1]), axis=-1) + .5) / np.array([xdivs, ydivs])
    anchor_box_template = np.full((xdivs, ydivs, 13 + boxes_per_cell * 6), centroid_locations)
    anchor_box_template[..., 14::6] = anchor_box_template[..., 14::6] + knudge_coords[..., 0:1]
    anchor_box_template[..., 15::6] = anchor_box_template[..., 15::6] + knudge_coords[..., 1:]

    return anchor_box_template

In [ ]:
arr = np.asarray(np.fromfunction(lambda x, y: [x, y], (12, 9), dtype=np.float32), dtype=np.float32)
dots = (np.stack((arr[0], arr[1]), axis=-1) + .5) / np.array([12, 9])

In [ ]:
divs = (9, 12) 
n_labels = train.get_labels(divs=divs, num_boxes=3, num_classes=13, normalized=True)
labels = train.get_labels(divs=divs, num_boxes=3, num_classes=13)
batch, xdiv, ydiv, _ = labels.shape

box_cutter = BoundingBox_Processor()
batch = n_labels.shape[0]

anchors = generate_anchors(n_labels, random_state=1)
anchor_tensor = np.full((batch,) + anchors.shape, anchors)

a_boxes = box_cutter.get_corners(anchor_tensor)
fig, ax = plt.subplots(figsize=(8,6))
ax.set(
        ylim=[0, 384],
        xlim=[0, 512],
        xticks=list(range(0, 512,int(512/12))),
        yticks=list(range(0, 384, int(384/9))),
        )
ax.grid(visible=True, zorder=0)
ax.set_title("Example of Calculated Shape Intersection")
ax.imshow(np.asarray(load_img("./data/images/train/screws_002.png", target_size=(384, 512))), zorder=0, alpha=.6)
for i in range(3):
    ax.add_patch(Polygon(box_cutter.get_corners(labels)[0][0, 5, 6], fill=None, edgecolor='chartreuse', lw=1, zorder=20))
    ax.add_patch(Polygon(a_boxes[i][0,5,6] * np.array([512, 384]), fill=None, edgecolor='fuchsia', lw=1, zorder=20))
for i in range(12 * 9):
    ax.add_patch(Circle(dots.reshape((12 * 9, 2))[i] * np.array([512, 384]), radius=3))
plt.savefig("./images/bbox_intersection.png")
plt.show()

In [ ]:
print(f"n_labels: {n_labels[0, 5, 6]}")
print(f"anchor_tensor: {anchor_tensor[0, 5, 6]}")

In [ ]:
int(divmod(.5, 1/12)[0])
int(divmod(.65, 1/9)[0])

In [ ]:
class AnchorLayer(Layer):
    def __init__(self, *, units, classes, boxes, anchors):
        super(AnchorLayer, self).__init__()
        self.units = units
        self.classes = classes
        self.boxes = boxes
        self.anchors = anchors
        self.box_cutter = BoundingBox_Processor()

    def build(self, input_shape):
        self.w = self.add_weight(
                shape=input_shape[1:-1] + (self.units * self.boxes,),
                initializer="random_normal",
                trainable=True,
                )
        self.b = self.add_weight(
                shape=(self.boxes * self.units,),
                initializer="random_normal",
                trainable=True,
                )

    def call(self, inputs):
        x = inputs[..., 13:]
        for i in range(self.boxes):
            anchor = anchors[..., self.boxes*i:self.boxes + i * self.boxes])
            self.box_cutter.IoU(x, anchor)

In [ ]:
anchor_layer = AnchorLayer(units=6, classes=13, boxes=3, anchors=None)
w, b = anchor_layer(labels)
print(f"w: {w.shape} | b: {b.shape}")